# Neuromatch Academy: Week 1, Day 5, Tutorial 1
# Dimensionality Reduction: Geometric view of data

__Content creators:__ Alex Cayco Gajic, John Murray

__Content reviewers:__ Roozbeh Farhoudi, Matt Krause, Spiros Chavlis, Richard Gao, Michael Waskom


---
# Tutorial Objectives

In this notebook we'll explore how multivariate data can be represented in different orthonormal bases. This will help us build intuition that will be helpful in understanding PCA in the following tutorial. 

Overview:
 - Generate correlated multivariate data.
 - Define an arbitrary orthonormal basis. 
 - Project the data onto the new basis.

In [ ]:
# @title Video 1: Geometric view of data
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="THu9yHnpq9I", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)
video

# Summary of Video 1:

- first tutorial: conceptualize multivariate data from a geometric point of view (useful for understanding how principal component analysis works)
- second tutorial: PCA
- third tutorial: use PCA for dimensionality reduction in high dimensional data.
- final tutorial: nonlinear dimensionality reduction technique called t-SNE.

Geometric view of data:
- Multivariate means many variables 
- each Xi represents the firing rate of a particular neuron, from the whole population of neurons that you're recording from.
(each Xi could represent a behavioral variable for a particular behavioral task.)
So at each time point in the experiment, collect a new vector corresponding to the firing rates of all of these neurons. 

Data representation:
- One way is to just stack all of the vectors together into the data matrix X. In this matrix, each row corresponds to a single sample of all of the variables. So all of the firing rates of all neurons at a single time point. And each column corresponds to all of the samples of a single variable 
- Another way to represent this data is as a point in a high dimensional vector space described by the coordinate axes each of which represents the firing rates for a different neuron.

- get multiple data points => will lie on different locations in this vector space and that's because there is typically a lot of variability in neural data. 
But there's different flavors of variability, and one possibility is you may see data that looks uncorrelated (~variability of neuron 1 is pretty much the same as the variability of neuron 2). On the other hand, you might see uncorrelated data where one neuron has much higher variability than the other neuron.
On the other hand, you might see that your data is very correlated. And in this case notice that the direction corresponding to most of the variance in the data is
aligned neither with neuron 1's coordinate axis nor with axis of neuron 2. Essentially the goal of PCA is to find this direction.

Typically in neural data, even when we're recording from very large numbers of neurons. We don't observe that the neural activity patterns 
are uniformly distributed across the entire vector space of possible activity patterns. Usually we see that they are constrained to lie on a lower dimensional subspace,
and the goal of PCA is to find this low dimensional subspace by looking for directions of maximum variance in the data. 

---
# Setup

In [ ]:
# Import
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# @title Figure Settings
import ipywidgets as widgets  # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

In [ ]:
# @title Helper functions


def get_data(cov_matrix):
  """
  Returns a matrix of 1000 samples from a bivariate, zero-mean Gaussian.

  Note that samples are sorted in ascending order for the first random variable

  Args:
    cov_matrix (numpy array of floats): desired covariance matrix

  Returns:
    (numpy array of floats) : samples from the bivariate Gaussian, with each
                              column corresponding to a different random
                              variable
  """

  mean = np.array([0, 0])
  X = np.random.multivariate_normal(mean, cov_matrix, size=1000)
  indices_for_sorting = np.argsort(X[:, 0])
  X = X[indices_for_sorting, :]

  return X


def plot_data(X):
  """
  Plots bivariate data. Includes a plot of each random variable, and a scatter
  plot of their joint activity. The title indicates the sample correlation
  calculated from the data.

  Args:
    X (numpy array of floats) :   Data matrix each column corresponds to a
                                  different random variable

  Returns:
    Nothing.
  """

  fig = plt.figure(figsize=[8, 4])
  gs = fig.add_gridspec(2, 2)
  ax1 = fig.add_subplot(gs[0, 0])
  ax1.plot(X[:, 0], color='k')
  plt.ylabel('Neuron 1')
  plt.title('Sample var 1: {:.1f}'.format(np.var(X[:, 0])))
  ax1.set_xticklabels([])
  ax2 = fig.add_subplot(gs[1, 0])
  ax2.plot(X[:, 1], color='k')
  plt.xlabel('Sample Number')
  plt.ylabel('Neuron 2')
  plt.title('Sample var 2: {:.1f}'.format(np.var(X[:, 1])))
  ax3 = fig.add_subplot(gs[:, 1])
  ax3.plot(X[:, 0], X[:, 1], '.', markerfacecolor=[.5, .5, .5],
           markeredgewidth=0)
  ax3.axis('equal')
  plt.xlabel('Neuron 1 activity')
  plt.ylabel('Neuron 2 activity')
  plt.title('Sample corr: {:.1f}'.format(np.corrcoef(X[:, 0], X[:, 1])[0, 1]))
  plt.show()


def plot_basis_vectors(X, W):
  """
  Plots bivariate data as well as new basis vectors.

  Args:
    X (numpy array of floats) :   Data matrix each column corresponds to a
                                  different random variable
    W (numpy array of floats) :   Square matrix representing new orthonormal
                                  basis each column represents a basis vector

  Returns:
    Nothing.
  """

  plt.figure(figsize=[4, 4])
  plt.plot(X[:, 0], X[:, 1], '.', color=[.5, .5, .5], label='Data')
  plt.axis('equal')
  plt.xlabel('Neuron 1 activity')
  plt.ylabel('Neuron 2 activity')
  plt.plot([0, W[0, 0]], [0, W[1, 0]], color='r', linewidth=3,
           label='Basis vector 1')
  plt.plot([0, W[0, 1]], [0, W[1, 1]], color='b', linewidth=3,
           label='Basis vector 2')
  plt.legend()
  plt.show()


def plot_data_new_basis(Y):
  """
  Plots bivariate data after transformation to new bases.
  Similar to plot_data but with colors corresponding to projections onto
  basis 1 (red) and basis 2 (blue). The title indicates the sample correlation
  calculated from the data.

  Note that samples are re-sorted in ascending order for the first
  random variable.

  Args:
    Y (numpy array of floats):   Data matrix in new basis each column
                                 corresponds to a different random variable

  Returns:
    Nothing.
  """
  fig = plt.figure(figsize=[8, 4])
  gs = fig.add_gridspec(2, 2)
  ax1 = fig.add_subplot(gs[0, 0])
  ax1.plot(Y[:, 0], 'r')
  plt.xlabel
  plt.ylabel('Projection \n basis vector 1')
  plt.title('Sample var 1: {:.1f}'.format(np.var(Y[:, 0])))
  ax1.set_xticklabels([])
  ax2 = fig.add_subplot(gs[1, 0])
  ax2.plot(Y[:, 1], 'b')
  plt.xlabel('Sample number')
  plt.ylabel('Projection \n basis vector 2')
  plt.title('Sample var 2: {:.1f}'.format(np.var(Y[:, 1])))
  ax3 = fig.add_subplot(gs[:, 1])
  ax3.plot(Y[:, 0], Y[:, 1], '.', color=[.5, .5, .5])
  ax3.axis('equal')
  plt.xlabel('Projection basis vector 1')
  plt.ylabel('Projection basis vector 2')
  plt.title('Sample corr: {:.1f}'.format(np.corrcoef(Y[:, 0], Y[:, 1])[0, 1]))
  plt.show()

# Discussion of Helper functions:

*get_data*:
takes the covariance matrix and returns the data matrix (1000 samples from bivariate, zero mean gaussian - each column corresponding to different random variable) sorted in ascending order for the first random variable.  

*plot_data*:
Plots bivariate data. Includes a plot of each random variable neuron1 (ylabel) and neuron2 (ylabel) with sample number (xaxis) : a scatter
  plot of their activity indicating the sample correlation calculated from the data.

*plot_basis_vectors*:
plots bivariable data with new basis vectors using orthonormal basis matrix (each column represents basis vector) and data matrix where each column corresponds to different random variable.

*plot_data_new_basis*:
Plots bivariate data after transformation to new bases with colors corresponding to projections onto basis 1 and basis 2 indicating the sample correlation calculated from the data matrix in new basis with each column corresponding to a different random variable. Samples are re-sorted in ascending order for the first random variable.

---
# Section 1: Generate correlated multivariate data

In [ ]:
# @title Video 2: Multivariate data
video = YouTubeVideo(id="jcTq2PgU5Vw", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)
video

# Summary of Video 2:

To measure the variability of each neuron individually, we can use the variance, which describes the spread of the distribution of firing rates around it's average value.

if you measure the firing rates in Hertz, then the covariance would be in Hertz squared. So you can get rid of this, by dividing by the product of the standard deviations and this also normalizes. We call this the correlation coefficient.
All of this information can be summarized in the covariance matrix, which is usually called capital sigma. This is an N by N matrix whose ij element is the covariance of xi and xj. That means that the ji element is also the covariance of xi and xj. So the covariance matrix is symmetric (sigma is equal to its own transpose) across its diagonal and the ii element is given by the variance.

This affords the covariance matrix with some important quantities that will be used in PCA.
In the first exercise model neural data as coming from a multivariate normal distribution. This is a generalization of a one-dimensional normal distribution to N dimensions with a given covariance structure. And if you look at the marginal distribution of each neurons separately, then you recover the original one dimensional Gaussian. 



To gain intuition, we will first use a simple model to generate multivariate data. Specifically, we will draw random samples from a *bivariate normal distribution*. This is an extension of the one-dimensional normal distribution to two dimensions, in which each $x_i$ is marginally normal with mean $\mu_i$ and variance $\sigma_i^2$:

\begin{align}
x_i \sim \mathcal{N}(\mu_i,\sigma_i^2).
\end{align}

Additionally, the joint distribution for $x_1$ and $x_2$ has a specified correlation coefficient $\rho$. Recall that the correlation coefficient is a normalized version of the covariance, and ranges between -1 and +1:

\begin{align}
\rho = \frac{\text{cov}(x_1,x_2)}{\sqrt{\sigma_1^2 \sigma_2^2}}.
\end{align}

For simplicity, we will assume that the mean of each variable has already been subtracted, so that $\mu_i=0$. The remaining parameters can be summarized in the covariance matrix, which for two dimensions has the following form:

\begin{equation*}
{\bf \Sigma} = 
\begin{pmatrix}
 \text{var}(x_1) & \text{cov}(x_1,x_2) \\
 \text{cov}(x_1,x_2) &\text{var}(x_2)
\end{pmatrix}.
\end{equation*}

In general, $\bf \Sigma$ is a symmetric matrix with the variances $\text{var}(x_i) = \sigma_i^2$ on the diagonal, and the covariances on the off-diagonal. Later, we will see that the covariance matrix plays a key role in PCA.




## Exercise 1: Draw samples from a distribution

We have provided code to draw random samples from a zero-mean bivariate normal distribution. Throughout this tutorial, we'll imagine these samples represent the activity (firing rates) of two recorded neurons on different trials. Fill in the function below to calculate the covariance matrix given the desired variances and correlation coefficient. The covariance can be found by rearranging the equation above:

\begin{align}
\text{cov}(x_1,x_2) = \rho \sqrt{\sigma_1^2 \sigma_2^2}.
\end{align}

Use these functions to generate and plot data while varying the parameters. You should get a feel for how changing the correlation coefficient affects the geometry of the simulated data.

**Steps**
* Fill in the function `calculate_cov_matrix` to calculate the desired covariance.
* Generate and plot the data for $\sigma_1^2 =1$, $\sigma_1^2 =1$, and $\rho = .8$. Try plotting the data for different values of the correlation coefficient: $\rho = -1, -.5, 0, .5, 1$.

In [ ]:
help(plot_data)
help(get_data)

In [ ]:
def calculate_cov_matrix(var_1, var_2, corr_coef):
  """
  Calculates the covariance matrix based on the variances and correlation
  coefficient.

  Args:
    var_1 (scalar)          : variance of the first random variable
    var_2 (scalar)          : variance of the second random variable
    corr_coef (scalar)      : correlation coefficient

  Returns:
    (numpy array of floats) : covariance matrix
  """

  #################################################
  ## TODO for students: calculate the covariance matrix
  # Fill out function and remove
  raise NotImplementedError("Student excercise: calculate the covariance matrix!")
  #################################################

  # Calculate the covariance from the variances and correlation
  cov = ...

  cov_matrix = np.array([[var_1, cov], [cov, var_2]])

  return cov_matrix


###################################################################
## TO DO for students: generate and plot bivariate Gaussian data with variances of 1
## and a correlation coefficients of: 0.8
## repeat while varying the correlation coefficient from -1 to 1
###################################################################
np.random.seed(2020)  # set random seed
variance_1 = 1
variance_2 = 1
corr_coef = 0.8

# Uncomment to test your code and plot
# cov_matrix = calculate_cov_matrix(variance_1, variance_2, corr_coef)
# X = get_data(cov_matrix)
# plot_data(X)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D5_DimensionalityReduction/solutions/W1D5_Tutorial1_Solution_a068bc84.py)

*Example output:*

<img alt='Solution hint' align='left' width=570 height=272 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W1D5_DimensionalityReduction/static/W1D5_Tutorial1_Solution_a068bc84_0.png>



---
# Section 2: Define a new orthonormal basis


In [ ]:
# @title Video 3: Orthonormal bases
video = YouTubeVideo(id="PC1RZELnrIg", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)
video

# Summary of Video 3:

Remember that the goal for dimensionality reduction is to find a lower dimensional subspace that you can use to capture much of the structure of your original data.
We need to understand how you can represent data in multiple different coordinate systems.

In fact there are many ways to represent multivariate data.
We can define a basis as a set of N vectors that you can use to construct any point in an N-dimensional vector space.
(specific choice of basis called the standard basis in which your first basis vector is one unit in the direction of the
activity for neuron one and the second basis vector is one unit in the direction of neuron two.)
One could rotate these basis vectors and therefore represent this point as a completely different set of coefficients.
You can also rescale your basis 

By orthonormal, we mean that the basis is orthogonal and all basis vectors have a length of 1, 
and we can measure the length as the square root of the sum of the squared values of the entries.
It's worth noting that if you already have an orthogonal basis, then you can very easily normalize it by
dividing each basis vector by its magnitude.

So if they're perpendicular or orthogonal then the cosine is equal to zero,
so that means that the dot product is zero. And in fact two vectors are defined as orthogonal
exactly when the dot product is equal to zero.

Another way to find if two vectors are orthogonal - 
- u dot w can also be written as the sum of ui times wi.
- So if u and w are both column vectors then u dot w is u transpose times w.

Next, we will define a new orthonormal basis of vectors ${\bf u} = [u_1,u_2]$ and ${\bf w} = [w_1,w_2]$. As we learned in the video, two vectors are orthonormal if: 

1.   They are orthogonal (i.e., their dot product is zero):
\begin{equation}
{\bf u\cdot w} = u_1 w_1 + u_2 w_2 = 0
\end{equation}
2.   They have unit length:
\begin{equation}
||{\bf u} || = ||{\bf w} || = 1
\end{equation}

In two dimensions, it is easy to make an arbitrary orthonormal basis. All we need is a random vector ${\bf u}$, which we have normalized. If we now define the second basis vector to be ${\bf w} = [-u_2,u_1]$, we can check that both conditions are satisfied: 
\begin{equation}
{\bf u\cdot w} = - u_1 u_2 + u_2 u_1 = 0
\end{equation}
and 
\begin{equation}
{|| {\bf w} ||} = \sqrt{(-u_2)^2 + u_1^2} = \sqrt{u_1^2 + u_2^2} = 1,
\end{equation}
where we used the fact that ${\bf u}$ is normalized. So, with an arbitrary input vector, we can define an orthonormal basis, which we will write in matrix by stacking the basis vectors horizontally:

\begin{equation}
{{\bf W} } =
\begin{pmatrix}
 u_1 & w_1 \\
 u_2 & w_2
\end{pmatrix}.
\end{equation}

 

## Exercise 2: Find an orthonormal basis

In this exercise you will fill in the function below to define an orthonormal basis, given a single arbitrary 2-dimensional vector as an input.

**Steps**
* Modify the function `define_orthonormal_basis` to first normalize the first basis vector $\bf u$.
* Then complete the function by finding a basis vector $\bf w$ that is orthogonal to $\bf u$.
* Test the function using initial basis vector ${\bf u} = [3,1]$. Plot the resulting basis vectors on top of the data scatter plot using the function `plot_basis_vectors`. (For the data, use  $\sigma_1^2 =1$, $\sigma_2^2 =1$, and $\rho = .8$).

In [ ]:
help(plot_basis_vectors)

In [ ]:
def define_orthonormal_basis(u):
  """
  Calculates an orthonormal basis given an arbitrary vector u.

  Args:
    u (numpy array of floats) : arbitrary 2-dimensional vector used for new
                                basis

  Returns:
    (numpy array of floats)   : new orthonormal basis
                                columns correspond to basis vectors
  """

  #################################################
  ## TODO for students: calculate the orthonormal basis
  # Fill out function and remove
  raise NotImplementedError("Student excercise: implement the orthonormal basis function")
  #################################################

  # normalize vector u
  u = ...
  # calculate vector w that is orthogonal to w
  w = ...

  W = np.column_stack([u, w])

  return W


np.random.seed(2020)  # set random seed
variance_1 = 1
variance_2 = 1
corr_coef = 0.8

cov_matrix = calculate_cov_matrix(variance_1, variance_2, corr_coef)
X = get_data(cov_matrix)
u = np.array([3, 1])

# Uncomment and run below to plot the basis vectors
# W = define_orthonormal_basis(u)
# plot_basis_vectors(X, W)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D5_DimensionalityReduction/solutions/W1D5_Tutorial1_Solution_85e5b50f.py)

*Example output:*

<img alt='Solution hint' align='left' width=274 height=272 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W1D5_DimensionalityReduction/static/W1D5_Tutorial1_Solution_85e5b50f_0.png>



---
# Section 3: Project data onto new basis

In [ ]:
# @title Video 4: Change of basis
video = YouTubeVideo(id="Mj6BRQPKKUc", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)
video

#Summary of Video 4:

how can you transform from your old basis to a new basis?
- easy if we have a new orthonormal basis. 
- project and find magnitude of that projection (hypotenuse and its' adjacent).
So that boils down to the magnitude of projected_vector times the cosine of theta or dot product if vector you project onto is normalised.
So for an orthonormal basis your new coordinates are the old coordinates dot the new basis vector. 


   
Finally, we will express our data in the new basis that we have just found. Since $\bf W$ is orthonormal, we can project the data into our new basis using simple matrix multiplication :

\begin{equation}
{\bf Y = X W}.
\end{equation}

We will explore the geometry of the transformed data $\bf Y$ as we vary the choice of basis.

## Exercise 3: Define an orthonormal basis
In this exercise you will fill in the function below to define an orthonormal basis, given a single arbitrary vector as an input.

**Steps**
* Complete the function `change_of_basis` to project the data onto the new basis.
* Plot the projected data using the function `plot_data_new_basis`. 
* What happens to the correlation coefficient in the new basis? Does it increase or decrease? 
* What happens to variance? 



In [ ]:
def change_of_basis(X, W):
  """
  Projects data onto new basis W.

  Args:
    X (numpy array of floats) : Data matrix each column corresponding to a
                                different random variable
    W (numpy array of floats) : new orthonormal basis columns correspond to
                                basis vectors

  Returns:
    (numpy array of floats)    : Data matrix expressed in new basis
  """

  #################################################
  ## TODO for students: project the data onto o new basis W
  # Fill out function and remove
  raise NotImplementedError("Student excercise: implement change of basis")
  #################################################

  # project data onto new basis described by W
  Y = ...

  return Y


# Unomment below to transform the data by projecting it into the new basis
# Y = change_of_basis(X, W)
# plot_data_new_basis(Y)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D5_DimensionalityReduction/solutions/W1D5_Tutorial1_Solution_a1124bbc.py)

*Example output:*

<img alt='Solution hint' align='left' width=560 height=272 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W1D5_DimensionalityReduction/static/W1D5_Tutorial1_Solution_a1124bbc_0.png>



## Interactive Demo: Play with the basis vectors
To see what happens to the correlation as we change the basis vectors, run the cell below. The parameter $\theta$ controls the angle of $\bf u$ in degrees. Use the slider to rotate the basis vectors. 

In [ ]:
# @title

# @markdown Make sure you execute this cell to enable the widget!


def refresh(theta=0):
  u = [1, np.tan(theta * np.pi / 180)]
  W = define_orthonormal_basis(u)
  Y = change_of_basis(X, W)
  plot_basis_vectors(X, W)
  plot_data_new_basis(Y)


_ = widgets.interact(refresh, theta=(0, 90, 5))

## Questions

* What happens to the projected data as you rotate the basis? 
* How does the correlation coefficient change? How does the variance of the projection onto each basis vector change?
* Are you able to find a basis in which the projected data is **uncorrelated**?

*hint*:
1. think of effect on sample correlation value. 
2. think of relationship between variance, correlation matrix, and theta. 
3. think of covariance values that cause projected data to be uncorrelated

---
# Summary

- In this tutorial, we learned that multivariate data can be visualized as a cloud of points in a high-dimensional vector space. The geometry of this cloud is shaped by the covariance matrix.

- Multivariate data can be represented in a new orthonormal basis using the dot product. These new basis vectors correspond to specific mixtures of the original variables - for example, in neuroscience, they could represent different ratios of activation  across a population of neurons.

- The projected data (after transforming into the new basis) will generally have a different geometry from the original data. In particular, taking basis vectors that are aligned with the spread of cloud of points decorrelates the data.

* These concepts - covariance, projections, and orthonormal bases - are key for understanding PCA, which will be our focus in the next tutorial.